In [ ]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from joblib import dump, load
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [ ]:
# paths to data
data_dir = '/kaggle/input/fashion-iq-dataset/fashionIQ_dataset'
image_dir = os.path.join(data_dir, 'images')
json_dir = os.path.join(data_dir, 'image_splits')

In [ ]:
# function to read data files

# read data in json file
def read_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

#  load image (path of image) from file json (follow cattegory)
def load_image_list(category, split):
    json_path = os.path.join(json_dir, f'split.{category}.{split}.json')
    image_list = read_json(json_path)
    return [os.path.join(image_dir, image_name + '.jpg') for image_name in image_list]

# function to preprocess data

# define transform for preprocess step
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# define class FashionIQDataset 
class FashionIQDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# load and split data in corresponding set 

# load images follow their category in three set: train, val and test
# category - dress
train_images_dress = load_image_list('dress', 'train')
val_images_dress = load_image_list('dress', 'val')
test_images_dress = load_image_list('dress', 'test')

# category - shirt
train_images_shirt = load_image_list('shirt', 'train')
val_images_shirt = load_image_list('shirt', 'val')
test_images_shirt = load_image_list('shirt', 'test')

# category - top&tee
train_images_toptee = load_image_list('toptee', 'train')
val_images_toptee = load_image_list('toptee', 'val')
test_images_toptee = load_image_list('toptee', 'test')

# combine all image (in each set) to make a dataset
train_images = train_images_dress + train_images_shirt + train_images_toptee
val_images = val_images_dress + val_images_shirt + val_images_toptee
test_images = test_images_dress + test_images_shirt + test_images_toptee

# label for images (0: dress, 1: shirt, 2: toptee)
train_labels = [0] * len(train_images_dress) + [1] * len(train_images_shirt) + [2] * len(train_images_toptee)
val_labels = [0] * len(val_images_dress) + [1] * len(val_images_shirt) + [2] * len(val_images_toptee)
test_labels = [0] * len(test_images_dress) + [1] * len(test_images_shirt) + [2] * len(test_images_toptee)

In [ ]:
# pre-process data
train_dataset = FashionIQDataset(train_images, train_labels, transform=transform)
val_dataset = FashionIQDataset(val_images, val_labels, transform=transform)
test_dataset = FashionIQDataset(test_images, test_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# if device is GPU 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_dir = '/kaggle/input/resnet50-weight-fashion/best_model.pth'
num_classes = 3

# init resnet50 model as the way training
model = models.resnet50(weights=None)  # pretrained=False vì bạn sẽ tải trọng số của mình
model.fc = nn.Linear(model.fc.in_features, num_classes)

# load 
# model.load_state_dict(torch.load(model_dir))
model.load_state_dict(torch.load(model_dir))

model.to(device)  
# Chuyển mô hình về chế độ đánh giá (evaluation mode)
model.eval()

In [ ]:
def extract_features_batchwise(loader, model, device, save_path_features, save_path_labels):
    model.eval()
    features_list = []
    labels_list = []
    
    os.makedirs(os.path.dirname(save_path_features), exist_ok=True)
    os.makedirs(os.path.dirname(save_path_labels), exist_ok=True)
    
    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(loader):
            images = images.to(device)
            output = model(images)
            output = output.view(output.size(0), -1)
            features_list.append(output.cpu().numpy())
            labels_list.append(labels.numpy())
            
            # save feature in batch to save resource
            if batch_idx % 20 == 0:  # save after 10 batch
                batch_features = np.concatenate(features_list, axis=0)
                batch_labels = np.concatenate(labels_list, axis=0)
                np.save(f'{save_path_features}_batch_{batch_idx}.npy', batch_features, allow_pickle=False)
                np.save(f'{save_path_labels}_batch_{batch_idx}.npy', batch_labels, allow_pickle=False)
                features_list = []
                labels_list = []
            
    if features_list:
        batch_features = np.concatenate(features_list, axis=0)
        batch_labels = np.concatenate(labels_list, axis=0)
        np.save(f'{save_path_features}_final.npy', batch_features, allow_pickle=False)
        np.save(f'{save_path_labels}_final.npy', batch_labels, allow_pickle=False)
        
        
def merge_npy_files(folder_path, file_pattern, output_file):
    sorted_files = sorted(
    [f for f in os.listdir(folder_path) if f.startswith(file_pattern + '_batch') and f.endswith('.npy')],
    key=lambda x: int(x.split('_')[-1].split('.')[0])
)
    sorted_files.append(file_pattern + '_final.npy')
    arrays = [np.load(os.path.join(folder_path, f)) for f in sorted_files]
    
    all_array = np.concatenate(arrays, axis=0)
    np.save(output_file, all_array, allow_pickle=False)
    print(f"Saved combined features to {output_file}")

In [ ]:
save_path_features = '/kaggle/working/features/features/'
save_path_labels = '/kaggle/working/features/labels/'

extract_features_batchwise(train_loader, model, device,save_path_features + 'train/features', save_path_labels + 'train/labels')
extract_features_batchwise(val_loader, model, device,save_path_features + 'valid/features', save_path_labels + 'valid/labels')
extract_features_batchwise(test_loader, model, device,save_path_features + 'test/features', save_path_labels + 'test/labels')

merge_npy_files('/kaggle/working/features/labels/train', 'labels', '/kaggle/working/features/train_labels.npy')
merge_npy_files('/kaggle/working/features/features/train', 'features', '/kaggle/working/features/train_features.npy')

merge_npy_files('/kaggle/working/features/labels/valid', 'labels', '/kaggle/working/features/val_labels.npy')
merge_npy_files('/kaggle/working/features/features/valid', 'features', '/kaggle/working/features/val_features.npy')

merge_npy_files('/kaggle/working/features/labels/test', 'labels', '/kaggle/working/features/test_labels.npy')
merge_npy_files('/kaggle/working/features/features/test', 'features', '/kaggle/working/features/test_features.npy')

In [ ]:
X_train = np.load('/kaggle/input/features-and-labels-fashioniq/train_features.npy')
y_train = np.load('/kaggle/input/features-and-labels-fashioniq/train_labels.npy')

X_valid = np.load('/kaggle/input/features-and-labels-fashioniq/val_features.npy')
y_valid = np.load('/kaggle/input/features-and-labels-fashioniq/val_labels.npy')

X_test = np.load('/kaggle/input/features-and-labels-fashioniq/test_features.npy')
y_test = np.load('/kaggle/input/features-and-labels-fashioniq/test_labels.npy')

In [ ]:


# sử dụng để tìm kiếm siêu tham số tốt nhất cho mô hình
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']  # Chỉ áp dụng cho kernel phi tuyến
}

svm = SVC()

grid_search = GridSearchCV(svm, param_grid, cv=5, scoring = 'accuracy')

# Huấn luyện mô hình với train set và tinh chỉnh với validation set
print("Training data")
grid_search.fit(X_train, y_train)

# Kết quả hyperparameters tốt nhất
print("Best Parameters: ", grid_search.best_params_)

# Đánh giá trên validation set 
val_accuracy = grid_search.score(X_valid, y_valid)
print("Validation Accuracy: ", val_accuracy)

In [ ]:
from sklearn.metrics import accuracy_score

# Kết quả của GridSearchCV
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)
print("Best Estimator: ", grid_search.best_estimator_)

# Xem thông tin về tất cả các kết quả trong GridSearch
results = grid_search.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Mean Score: {mean_score:.3f} with parameters: {params}")


In [ ]:
# lưu trọng số đã huấn luyện
dump(grid_search, '/kaggle/working/svm_model.joblib')